In [ ]:
import module_loader
import pandas as pd
from modules.curves import *
from modules.dual import Dual

# Principal Component Analysis Methods

### Build a simple curve for risking and pricing

In [ ]:
nodes = {
    datetime(2022, 1, 1): Dual(1, {"v0": 1}),
    datetime(2024, 1, 1): Dual(1, {"v1": 1}),
    datetime(2027, 1, 1): Dual(1, {"v2": 1}),
    datetime(2032, 1, 1): Dual(1, {"v3": 1}),
    datetime(2052, 1, 1): Dual(1, {"v4": 1}),
}
swaps = {
    Swap(datetime(2022, 1, 1), 12*2, 12, 12): 1.20,
    Swap(datetime(2022, 1, 1), 12*5, 12, 12): 1.66,
    Swap(datetime(2022, 1, 1), 12*10, 12, 12): 1.93,
    Swap(datetime(2022, 1, 1), 12*30, 12, 12): 2.20,
}
s_cv = SolvedCurve(nodes=nodes, interpolation="log_linear", swaps=list(swaps.keys()), obj_rates=list(swaps.values()))
s_cv.iterate()  

### Add a short historical data sample for covariance

In [ ]:
historical_rates = pd.DataFrame({
    "2Y": [1.199, 1.228, 1.210, 1.215, 1.203, 1.159, 1.175, 1.188, 1.159, 1.100],
    "5Y": [1.663, 1.696, 1.665, 1.680, 1.677, 1.657, 1.673, 1.676, 1.653, 1.600],
    "10Y": [1.928, 1.945, 1.934, 1.93, 1.934, 1.931, 1.958, 1.972, 1.932, 1.900],
    "30Y": [2.201, 2.217, 2.228, 2.239, 2.226, 2.235, 2.242, 2.236, 2.22, 2.200],
})
historical_chgs = historical_rates.diff(-1)*100
Q = historical_chgs.cov().to_numpy()
Q

### Create a portfolio of swaps replicating the risks in the text

In [ ]:
portfolio = Portfolio(objects=[
    Swap(datetime(2022,1,1), 12*2, 12, 12, notional=-1.272e6),
    Swap(datetime(2022,1,1), 12*5, 12, 12, notional=313545),
    Swap(datetime(2022,1,1), 12*10, 12, 12, notional=109931),
    Swap(datetime(2022,1,1), 12*30, 12, 12, notional=27160),
])  

### Utilise all the PCA methods displaying results

In [ ]:
lambd, E = portfolio.pca(Q)
print(lambd)
print(E)

In [ ]:
portfolio.historical_multipliers(Q, historical_chgs)

In [ ]:
portfolio.risk(s_cv)

In [ ]:
portfolio.pca_risk(s_cv, Q)

In [ ]:
portfolio.pca_covar_alloc(s_cv, Q)